In [0]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main a

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import udf,pandas_udf,PandasUDFType 

In [0]:
df=spark.read.csv('zoo.csv',header=True)

In [0]:
spark_df=df

In [0]:
spark_df.show(3)

+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----------+
|animal_name|hair|feathers|eggs|milk|airborne|aquatic|predator|toothed|backbone|breathes|venomous|fins|legs|tail|domestic|catsize|class_type|
+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----------+
|   aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|         1|
|   antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|   4|   1|       0|      1|         1|
|       bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|   0|   1|       0|      0|         4|
+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----------+
only s

In [0]:
import pandas as pd
import numpy as np

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from sklearn.tree import DecisionTreeClassifier

In [0]:
spark_df.show(3)

+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----------+
|animal_name|hair|feathers|eggs|milk|airborne|aquatic|predator|toothed|backbone|breathes|venomous|fins|legs|tail|domestic|catsize|class_type|
+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----------+
|   aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|         1|
|   antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|   4|   1|       0|      1|         1|
|       bass|   0|       0|   1|   0|       0|      1|       1|      1|       1|       0|       0|   1|   0|   1|       0|      0|         4|
+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----------+
only s

In [0]:
schema=spark_df.schema

In [0]:
schema

StructType(List(StructField(animal_name,StringType,true),StructField(hair,StringType,true),StructField(feathers,StringType,true),StructField(eggs,StringType,true),StructField(milk,StringType,true),StructField(airborne,StringType,true),StructField(aquatic,StringType,true),StructField(predator,StringType,true),StructField(toothed,StringType,true),StructField(backbone,StringType,true),StructField(breathes,StringType,true),StructField(venomous,StringType,true),StructField(fins,StringType,true),StructField(legs,StringType,true),StructField(tail,StringType,true),StructField(domestic,StringType,true),StructField(catsize,StringType,true),StructField(class_type,StringType,true)))

In [0]:
import numpy as np
@pandas_udf(schema,PandasUDFType.GROUPED_MAP)
def applyModel(df):
  model=DecisionTreeClassifier()
  y=df['class_type']
  x=df.drop(['id','animal_name','class_type'],axis=1)
  print('~~~~~~X~~~~~~~')
  #print(x.head())
  print('~~~~~~Y~~~~~~~')
  #print(y.head())
  model.fit(x,y)
  preds=model.predict(testx)
  datadict={
      'actual':list(testx),
      'predicted':list(preds)
  }
  return pd.DataFrame(datadict)
  

In [0]:
spark_df.show(2)

+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----------+
|animal_name|hair|feathers|eggs|milk|airborne|aquatic|predator|toothed|backbone|breathes|venomous|fins|legs|tail|domestic|catsize|class_type|
+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----------+
|   aardvark|   1|       0|   0|   1|       0|      0|       1|      1|       1|       1|       0|   0|   4|   0|       0|      1|         1|
|   antelope|   1|       0|   0|   1|       0|      0|       0|      1|       1|       1|       0|   0|   4|   1|       0|      1|         1|
+-----------+----+--------+----+----+--------+-------+--------+-------+--------+--------+--------+----+----+----+--------+-------+----------+
only showing top 2 rows



In [0]:
sample_df=spark_df.select('hair','toothed','class_type')

In [0]:
sample_df.count()

101

In [0]:
sample_df.printSchema()

root
 |-- hair: string (nullable = true)
 |-- toothed: string (nullable = true)
 |-- class_type: string (nullable = true)



In [0]:
sample_df=sample_df.withColumn('hair',sample_df.hair.cast('integer'))
sample_df=sample_df.withColumn('toothed',sample_df.toothed.cast('integer'))
sample_df=sample_df.withColumn('class_type',sample_df.class_type.cast('integer'))
sample_df=sample_df.withColumn('id',F.monotonically_increasing_id())
sample_df=sample_df.withColumn('id',sample_df.id.cast('integer'))

In [0]:
sample_df.show(20)

+----+-------+----------+---+
|hair|toothed|class_type| id|
+----+-------+----------+---+
|   1|      1|         1|  0|
|   1|      1|         1|  1|
|   0|      1|         4|  2|
|   1|      1|         1|  3|
|   1|      1|         1|  4|
|   1|      1|         1|  5|
|   1|      1|         1|  6|
|   0|      1|         4|  7|
|   0|      1|         4|  8|
|   1|      1|         1|  9|
|   1|      1|         1| 10|
|   0|      0|         2| 11|
|   0|      1|         4| 12|
|   0|      0|         7| 13|
|   0|      0|         7| 14|
|   0|      0|         7| 15|
|   0|      0|         2| 16|
|   1|      1|         1| 17|
|   0|      1|         4| 18|
|   0|      1|         1| 19|
+----+-------+----------+---+
only showing top 20 rows



In [0]:
sample_df.printSchema()

root
 |-- hair: integer (nullable = true)
 |-- toothed: integer (nullable = true)
 |-- class_type: integer (nullable = true)
 |-- id: integer (nullable = false)



In [0]:
schema=sample_df.schema

In [0]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()

In [0]:
dir(model)

['C',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_estimator_type',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_predict_proba_lr',
 'class_weight',
 'decision_function',
 'densify',
 'dual',
 'fit',
 'fit_intercept',
 'get_params',
 'intercept_scaling',
 'l1_ratio',
 'max_iter',
 'multi_class',
 'n_jobs',
 'penalty',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'random_state',
 'score',
 'set_params',
 'solver',
 'sparsify',
 'tol',
 'verbose',
 'warm_start']

In [0]:
import pickle
stringrep=pickle.dumps(model)

In [0]:
@pandas_udf(schema,PandasUDFType.GROUPED_MAP)
def applyLogit(df):
  id=df.id.iloc[0]
  num_instances=df.shape()
  x=df[['hair','toothed']]
  y=df[['class_type']]
  model.fit(x,y)
  params=pickle.dumps(modes)
  return pd.DataFrame({'num_instances':num_instances,'x':x,'y':y,'params':params})

In [0]:
sample_df.printSchema()

root
 |-- hair: integer (nullable = true)
 |-- toothed: integer (nullable = true)
 |-- class_type: integer (nullable = true)
 |-- id: integer (nullable = false)



In [0]:
result=sample_df.groupby('id').apply(applyLogit)

In [0]:
sample_df.show(2)

+----+-------+----------+---+
|hair|toothed|class_type| id|
+----+-------+----------+---+
|   1|      1|         1|  0|
|   1|      1|         1|  1|
+----+-------+----------+---+
only showing top 2 rows



In [0]:
schema

StructType(List(StructField(hair,IntegerType,true),StructField(toothed,IntegerType,true),StructField(class_type,IntegerType,true),StructField(id,IntegerType,false)))

In [0]:
@pandas_udf(schema,PandasUDFType.GROUPED_MAP)
def dropCol(pdf):
  return pdf.assign(toothed=pdf.hair*2+pdf.toothed)
result=sample_df.groupby('id').apply(dropCol)

In [0]:
result.count()

101

In [0]:
result.where(result.id==0).show()

+----+-------+----------+---+
|hair|toothed|class_type| id|
+----+-------+----------+---+
|   1|      3|         1|  0|
+----+-------+----------+---+



In [0]:
sample_df.show(10)

+----+-------+----------+---+
|hair|toothed|class_type| id|
+----+-------+----------+---+
|   1|      1|         1|  0|
|   1|      1|         1|  1|
|   0|      1|         4|  2|
|   1|      1|         1|  3|
|   1|      1|         1|  4|
|   1|      1|         1|  5|
|   1|      1|         1|  6|
|   0|      1|         4|  7|
|   0|      1|         4|  8|
|   1|      1|         1|  9|
+----+-------+----------+---+
only showing top 10 rows



In [0]:
schema=StructType([
                   StructField('id',IntegerType(),False),
                   
])
@pandas_udf(schema,PandasUDFType.GROUPED_MAP)
def train(pdf):
  ids=pdf.id.iloc[0]
  df=pd.DataFrame({'id':[ids]})
  return df

In [0]:
result=sample_df.groupby('id').apply(train)

In [0]:
result.count()

101

In [0]:
result.show()

+---+
| id|
+---+
| 31|
| 85|
| 65|
| 53|
| 78|
| 34|
| 81|
| 28|
| 76|
| 26|
| 27|
| 44|
| 12|
| 91|
| 22|
| 93|
| 47|
|  1|
| 52|
| 13|
+---+
only showing top 20 rows



In [0]:
sample_df.count()

101

In [0]:
df.head()

,hair,toothed,class_type,id
0,1,1,1,0
1,1,1,1,1
2,0,1,4,2
3,1,1,1,3
4,1,1,1,4
